# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [73]:
import pandas as pd

doc = []
with open('assets/dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [7]:
import regex as re
import pandas as pd

doc = []
with open('assets/dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)

tmp = []
ndf = {}
i=0
#code is to parse through each line of df and parse through various regex terms and create a dictionary with all of them 
#represented by variable i as the key, and the date as the value. using many try and except clauses to attempt many forms of regex matching

#print(df[80])
for line in df:
    try: 
        x = re.findall(r'((?:\d{1,2}[ /-])?(?:Jan|January|Feb|February|Mar|March|Apr|April|May|Jun|June|Jul|July|Aug|August|Sept|Sep|September|Oct|October|Nov|November|Dec|December)[ \,\./-]+(?:\d{1,2}[ \,\./-]+)?(?:\d{2,4}))', line)
        j = []
        if x == j:
            y = re.findall(r'(\d{1,2}[/-](?:\d{1,2}[/-])?\d{2,4})', line)
            if y == j:
                y = re.findall(r'((?:Jan|January|Feb|February|Mar|March|Apr|April|May|Jun|June|Jul|July|Aug|August|Sept|Sep|September|Oct|October|Nov|November|Dec|December)[ \,\./-]+(?:\d{1,2}[sthnd]+[ \,\./-]+)(?:\d{2,4}))', line)
                if y == j:
                    t = re.findall(r'\d{4}', line)
                    if t ==j:
                        raise ValueError
                    else: ndf[i]=t
                else: ndf[i] = y
            else: ndf[i]=y
        else: ndf[i]=x
        
    except ValueError:
        z = line
        ndf[i]= z      
    i += 1

print(df[271],ndf[271])
    
#This Code block starts the process of sorting and cleaning the data into a dictionary with MM/DD/YYYY format    
for k,v in ndf.items():
    if len(v) == 2:
       tmp.append((k,v))
#print(tmp)
df[272]
ndf[272] = ['Feb 1993']
ndf[248] = ['July 1995']
ndf[72] = ['7/11/77']
ndf[10] = ['5/11/85']
ndf[80] = ['6/29/81']
ndf[401] = ['12/2014']
i = 0

def convert_month(match_obj):
    if match_obj.group(1) is not None:
        return '01'
    if match_obj.group(2) is not None:
        return '02'
    if match_obj.group(3) is not None:
        return '03'
    if match_obj.group(4) is not None:
        return '04'
    if match_obj.group(5) is not None:
        return '05'
    if match_obj.group(6) is not None:
        return '06'
    if match_obj.group(7) is not None:
        return '07'
    if match_obj.group(8) is not None:
        return '08'
    if match_obj.group(9) is not None:
        return '09'
    if match_obj.group(10) is not None:
        return '10'
    if match_obj.group(11) is not None:
        return '11'
    if match_obj.group(12) is not None:
        return '12'

#looks for each pattern of date style, then separates into format Month | Day | Year
for k,v in ndf.items(): 
    #MM/DD/YYYY format, MM/YYYY
    x = re.findall(r'(\d{1,2}[/-](?:\d{1,2}[/-])?\d{2,4})', v[0])
    j = []
    if x == j:
        # DD MMM YYYY or MMM DD YYYY format
        y = re.findall(r'((?:\d{1,2})?[ /-]?(?:Jan|January|Feb|February|Mar|March|Apr|April|May|Jun|June|Jul|July|Aug|August|Sept|Sep|September|Oct|October|Nov|November|Dec|December)[ \,\./-]+(?:\d{1,2}[ \,\./-]+)?(?:\d{2,4}))', v[0])
        if y == j:
            #YYYY format
            t = re.findall(r'\d{4}', v[0])
            if t ==j:
                pass
            else:
                #adds missing pieces to make 1/1/YYYY format
                t = ['1', '1', t[0].strip()]
                ndf[i]=t
        else:
            #Splits/strips and sorts in correct MM/DD/YYYY format
            y = re.split(' |, |\. |-', y[0].strip())
            if len(y) == 2:
                y.append(y[1])
                y[1] = '1'
            if len(y[0]) < 3:
                temp = y[0]
                y[0] = y[1]
                y[1] = temp
            ndf[i] = y
    else: 
        #splits/strips and adds values to missing for /YY and MM/YYYY so it becomes MM/DD/YYYY format
        x = re.split('/|-', x[0].strip())
        if len(x) == 2:
            x.append(x[1])
            x[1] = '1'
        if len(x[2]) == 2:
            x[2] = '19'+x[2]
        ndf[i] = x    
    i += 1
print(ndf[0])
for k,v, in ndf.items():
    if len(v[0]) > 2:
        ndf[k] = (re.sub(r'''(January|Jan)|(February|Feb)|(March|Mar)|(April|Apr)|(May)|(June|Jun)|(July|Jul)|(August|Aug)|(September|Sept|Sep)|
                         |(October|Oct)|(November|Nov)|(December|Dec)''', convert_month, v[0])+'/'+v[1]+'/'+v[2])
        if len(v[2]) < 4:
            print(k)
    else:
        ndf[k] = v[0]+'/'+v[1]+'/'+v[2]
for k,v in ndf.items():
    ndf[k] = [pd.to_datetime(v, dayfirst = False, yearfirst = False, format = '%m/%d/%Y'), k]
#turn the dictionary I created into a DataFrame that will be organized by date, so I can get a series of the OSI (original Series Index)
Date_df = pd.DataFrame.from_dict(ndf, orient='index', columns = ['Date', 'OSI'])
Date_df
Sort_df = Date_df.sort_values(by='Date')
#Sort_df
Answer = Sort_df['OSI'].reset_index()
Answer= Answer['OSI']
Answer


.Spoke to sister Naomi Ely 708-810-7787 who reports he has been doing much better since he went to Dysart Clinic (he was drinking for a month leading up to this, his ammonia was high, and physicians were worried about early).  She feels his cognition is back to baseline, "100% better".  She says he has been successful in abstaining from substances as far as she knows, thinks a schedule is useful to him, doctor's appts etc.  Notes that he returned from LA in August 2008, gets bouts of "exhaustion" even in sobriety.  She denies ever witnessing any periods of manic behavior from patient.  Their father has dementia that started at age 84.  Notes patient is living with uncle in Black River Falls (uncle is 89), lived with sister 3 months who also takes care of her own father in Talladega.  She knows he is working on getting social security, subsidizing housing.  Stable situation with patient's girlfriend Nutt.Suicidal Behavior Hx of Suicidal Behavior: No
 ['August 2008']
['03', '25', '1993']

0        9
1       84
2        2
3       53
4       28
      ... 
495    231
496    141
497    186
498    161
499    413
Name: OSI, Length: 500, dtype: int64

In [11]:
def date_sorter():
    
    order = None
    import regex as re
    import pandas as pd

    doc = []
    with open('assets/dates.txt') as file:
        for line in file:
            doc.append(line)

    df = pd.Series(doc)

    tmp = []
    ndf = {}
    i=0
    #code is to parse through each line of df and parse through various regex terms and create a dictionary with all of them 
    #represented by variable i as the key, and the date as the value. using many try and except clauses to attempt many forms of regex matching

    #print(df[80])
    for line in df:
        try: 
            x = re.findall(r'((?:\d{1,2}[ /-])?(?:Jan|January|Feb|February|Mar|March|Apr|April|May|Jun|June|Jul|July|Aug|August|Sept|Sep|September|Oct|October|Nov|November|Dec|December)[ \,\./-]+(?:\d{1,2}[ \,\./-]+)?(?:\d{2,4}))', line)
            j = []
            if x == j:
                y = re.findall(r'(\d{1,2}[/-](?:\d{1,2}[/-])?\d{2,4})', line)
                if y == j:
                    y = re.findall(r'((?:Jan|January|Feb|February|Mar|March|Apr|April|May|Jun|June|Jul|July|Aug|August|Sept|Sep|September|Oct|October|Nov|November|Dec|December)[ \,\./-]+(?:\d{1,2}[sthnd]+[ \,\./-]+)(?:\d{2,4}))', line)
                    if y == j:
                        t = re.findall(r'\d{4}', line)
                        if t ==j:
                            raise ValueError
                        else: ndf[i]=t
                    else: ndf[i] = y
                else: ndf[i]=y
            else: ndf[i]=x

        except ValueError:
            z = line
            ndf[i]= z      
        i += 1

    #print(df[271],ndf[271])

    #This Code block starts the process of sorting and cleaning the data into a dictionary with MM/DD/YYYY format    
    for k,v in ndf.items():
        if len(v) == 2:
           tmp.append((k,v))
    #print(tmp)
    df[272]
    ndf[272] = ['Feb 1993']
    ndf[248] = ['July 1995']
    ndf[72] = ['7/11/77']
    ndf[10] = ['5/11/85']
    ndf[80] = ['6/29/81']
    ndf[401] = ['12/2014']
    i = 0

    def convert_month(match_obj):
        if match_obj.group(1) is not None:
            return '01'
        if match_obj.group(2) is not None:
            return '02'
        if match_obj.group(3) is not None:
            return '03'
        if match_obj.group(4) is not None:
            return '04'
        if match_obj.group(5) is not None:
            return '05'
        if match_obj.group(6) is not None:
            return '06'
        if match_obj.group(7) is not None:
            return '07'
        if match_obj.group(8) is not None:
            return '08'
        if match_obj.group(9) is not None:
            return '09'
        if match_obj.group(10) is not None:
            return '10'
        if match_obj.group(11) is not None:
            return '11'
        if match_obj.group(12) is not None:
            return '12'

    #looks for each pattern of date style, then separates into format Month | Day | Year
    for k,v in ndf.items(): 
        #MM/DD/YYYY format, MM/YYYY
        x = re.findall(r'(\d{1,2}[/-](?:\d{1,2}[/-])?\d{2,4})', v[0])
        j = []
        if x == j:
            # DD MMM YYYY or MMM DD YYYY format
            y = re.findall(r'((?:\d{1,2})?[ /-]?(?:Jan|January|Feb|February|Mar|March|Apr|April|May|Jun|June|Jul|July|Aug|August|Sept|Sep|September|Oct|October|Nov|November|Dec|December)[ \,\./-]+(?:\d{1,2}[ \,\./-]+)?(?:\d{2,4}))', v[0])
            if y == j:
                #YYYY format
                t = re.findall(r'\d{4}', v[0])
                if t ==j:
                    pass
                else:
                    #adds missing pieces to make 1/1/YYYY format
                    t = ['1', '1', t[0].strip()]
                    ndf[i]=t
            else:
                #Splits/strips and sorts in correct MM/DD/YYYY format
                y = re.split(' |, |\. |-', y[0].strip())
                if len(y) == 2:
                    y.append(y[1])
                    y[1] = '1'
                if len(y[0]) < 3:
                    temp = y[0]
                    y[0] = y[1]
                    y[1] = temp
                ndf[i] = y
        else: 
            #splits/strips and adds values to missing for /YY and MM/YYYY so it becomes MM/DD/YYYY format
            x = re.split('/|-', x[0].strip())
            if len(x) == 2:
                x.append(x[1])
                x[1] = '1'
            if len(x[2]) == 2:
                x[2] = '19'+x[2]
            ndf[i] = x    
        i += 1
    #print(ndf[0])
    for k,v, in ndf.items():
        if len(v[0]) > 2:
            ndf[k] = (re.sub(r'''(January|Jan)|(February|Feb)|(March|Mar)|(April|Apr)|(May)|(June|Jun)|(July|Jul)|(August|Aug)|(September|Sept|Sep)|
                             |(October|Oct)|(November|Nov)|(December|Dec)''', convert_month, v[0])+'/'+v[1]+'/'+v[2])
            if len(v[2]) < 4:
                print(k)
        else:
            ndf[k] = v[0]+'/'+v[1]+'/'+v[2]
    for k,v in ndf.items():
        ndf[k] = [pd.to_datetime(v, dayfirst = False, yearfirst = False, format = '%m/%d/%Y'), k]
    #turn the dictionary I created into a DataFrame that will be organized by date, so I can get a series of the OSI (original Series Index)
    Date_df = pd.DataFrame.from_dict(ndf, orient='index', columns = ['Date', 'OSI'])
    Date_df
    Sort_df = Date_df.sort_values(by='Date')
    #Sort_df
    Answer = Sort_df['OSI'].reset_index()
    order= Answer['OSI']
    return order # Your answer here

In [8]:
x = ['1995', '6', '97']
y=[]
for item in x:
    if len(item) == 1 or len(item) == 2:
        item = '19'+item
    y.append(item)
if len(x) ==3:
    x.append(x[2])
    x[2] = '1'
y

['1995', '196', '1997']

In [9]:
x = ['Mar-20-2009', 'Mar 20, 2009', 'March 20, 2009', 'Mar. 20, 2009', 'Mar 20 2009', '20 Mar 2009', '20 March 2009', '20 Mar. 2009', '20 March, 2009']
y =[]
x[0].strip()
#print(x)
#for item in x:
#    y.append(re.split(' |, |\. |-', item))
#print(y)

'Mar-20-2009'

In [10]:
v= ['1994']
t = re.findall(r'\d{4}', v[0])
if t ==j:
    pass
else: 
    t = ['1', '1', t[0]]
t
    

['1', '1', '1994']

In [45]:


test_d = {0:['03', '25', '1984'], 1:['Mar','25','1984'], 2:['March','25','1984'], 3:['June','25','1984'],4:['January','25','1984'],5:['Oct','25','1984']}
print(test_d)

test_d

    

{0: ['03', '25', '1984'], 1: ['Mar', '25', '1984'], 2: ['March', '25', '1984'], 3: ['June', '25', '1984'], 4: ['January', '25', '1984'], 5: ['Oct', '25', '1984']}


{0: ['03', '25', '1984'],
 1: '03/25/1984',
 2: '03/25/1984',
 3: '06/25/1984',
 4: '01/25/1984',
 5: 'Oct/25/1984'}

In [12]:
date_sorter()

0        9
1       84
2        2
3       53
4       28
      ... 
495    231
496    141
497    186
498    161
499    413
Name: OSI, Length: 500, dtype: int64